In [1]:
import os; os.chdir('../..') # changing to root directory of project
from config import config

In [2]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from utils.save_features import save_features 

In [3]:
df = pd.read_csv('data/processed/heart_disease.csv')

In [4]:
target = 'heart_disease'

# features selections
features = df.drop(target, axis = 1)

In [5]:
# saving features in json file along with their lebel and options
save_features(features, 'heart_disease')

Starting to process columns in feature_df...
Updating features config file
Succesfully updated features config file


In [6]:
# encoding categorical datas
# initializing encoder
encoder = OneHotEncoder(sparse_output = False)

# columns to encode
categorical_columns = features.select_dtypes(include = 'object').columns

# training encoder 
encoder.fit(features[categorical_columns])

# encoding
encoded_columns = pd.DataFrame(encoder.transform(features[categorical_columns]))

# naming columns
encoded_columns.columns = encoder.get_feature_names_out(categorical_columns)

# concating with original dataframe
features = pd.concat([features.drop(categorical_columns, axis = 1), encoded_columns], axis = 1)

# saving encoder to encode data from user
with open('models/heart_disease/encoder.pkl', 'wb') as fp: 
    pickle.dump(encoder, fp)

In [7]:
# Normalizing numerical features 
# initializing standared scaler
scaler = StandardScaler()

# features to scale
numerical_features = features.select_dtypes(include='number').columns

# training scaler
scaler.fit(features[numerical_features])

# scaling
features[numerical_features] = scaler.transform(features[numerical_features])

# saving model 
with open('models/heart_disease/scaler.pkl', 'wb') as fp:
    pickle.dump(scaler, fp)

In [8]:
# spliting train test data
X = features # features
y = df[target] # target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [9]:
# saving train data
X_train.to_csv('data/model_training/heart_disease/X_train.csv', index=False)
X_test.to_csv('data/model_training/heart_disease/X_test.csv', index=False)
y_train.to_csv('data/model_training/heart_disease/y_train.csv', index=False)
y_test.to_csv('data/model_training/heart_disease/y_test.csv', index=False)
